In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import KFold

In [ ]:
df = pd.read_csv('../input/tabular-playground-series-mar-2022/train.csv')
test = pd.read_csv('../input/tabular-playground-series-mar-2022/test.csv')

In [ ]:
df

In [ ]:
df['time'] = pd.to_datetime(df['time'],format='%Y/%m/%d %H:%M:%S')
df

In [ ]:
df['year'] = df['time'].dt.year
df['month'] = df['time'].dt.month_name()
df['weekday'] = df['time'].dt.day_name()
df['day'] = df['time'].dt.day
df['hour'] = df['time'].dt.hour
df['minute'] = df['time'].dt.minute
df

In [ ]:
train_dataset = df.drop(['row_id', 'time', 'year', 'congestion', 'month'], axis = 1)
train_dataset

In [ ]:
dummies = []
nummerical = []
for column in train_dataset.dtypes.index:
    if train_dataset.dtypes[column] == 'object':
        dummies.append(column)
    else:
        nummerical.append(column)

In [ ]:
print(nummerical)
dummies

In [ ]:
X = train_dataset
y = df['congestion']

In [ ]:
#model = CatBoostRegressor( custom_metric = 'MAE', eval_metric = 'MAE', cat_features = dummies, random_state = 228)
#cat = GridSearchCV(model, {'max_depth' : [9, 10, 11], 'iterations' : [1300, 1400, 1500]}, scoring = 'neg_mean_absolute_error', cv = KFold(5, shuffle = True, random_state = 228))
#cat.fit(X, y, verbose = 0)
#print(cat.best_params_)
#cat.best_score_
#max_depth = 8
#iterations = 1200
#max_depth = 10
#iterations = 1300

In [ ]:
cat = CatBoostRegressor(custom_metric = 'MAE', eval_metric = 'MAE', max_depth = 13, iterations = 1400, cat_features = dummies, random_state = 228)
cat.fit(X, y, verbose = 0)

In [ ]:
test['time'] = pd.to_datetime(test['time'],format='%Y/%m/%d %H:%M:%S')
test['year'] = test['time'].dt.year
test['month'] = test['time'].dt.month_name()
test['weekday'] = test['time'].dt.day_name()
test['day'] = test['time'].dt.day
test['hour'] = test['time'].dt.hour
test['minute'] = test['time'].dt.minute
test

In [ ]:
test_dataset = test.drop(['row_id', 'time', 'year', 'month'], axis = 1)
test_dataset

In [ ]:
congestion = cat.predict(test_dataset)
congestion = congestion.reshape(2340,)
np.unique(congestion)

In [ ]:
with open('./submission.csv', 'w') as csv_file:
    labels = ['row_id', 'congestion']
    print(','.join(labels), file = csv_file)
    for i,j in zip(test['row_id'],congestion):
        print(f'{i},{j}', file = csv_file)